In [1]:
import numpy as np
import open3d as o3d
import math
import back_surface as back

In [2]:
def duplicateSurface(numny_array_points):
    temp = []
    for i in numny_array_points:
        temp.append([i[0],i[1],i[2]+0.08])
    return np.asarray(temp)

def createPointCloud(points_list):
    temp = o3d.geometry.PointCloud()
    temp.points = o3d.utility.Vector3dVector(np.asarray(points_list))
    return temp

def getCenterPoint(points_list):
    count = len(points_list)
    #init
    center = np.zeros(3)
    for i in range(count):
        center[0] += points_list[i][0]
        center[1] += points_list[i][1]
        center[2] += points_list[i][2]
    return center/count

def computeNormal(points,cam):
    ##search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
    points.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=9))
    points.orient_normals_towards_camera_location(camera_location=cam)
    o3d.visualization.draw_geometries([points])

def mergeTwoClouds(fcloud,scloud):
    points = np.append(np.asarray(fcloud.points),np.asarray(scloud.points),axis=0)
    normals = np.append(np.asarray(fcloud.normals),np.asarray(scloud.normals),axis=0)
    result = createPointCloud(points)
    result.normals = o3d.utility.Vector3dVector(normals)
    return result

In [3]:
pcd = o3d.io.read_point_cloud("./Points/clean points/dwnsample_pose_1.ply")

In [18]:
# lật ngược các điểm
points = np.asarray(pcd.points)
behi = back.generateBack(points,0.05,0.1,0.07)
bcd = createPointCloud(behi) #back point cloud

In [19]:
cam_front = np.array([0,0,0])
cam_back = np.array([0,0,10])
computeNormal(pcd,cam_front)
computeNormal(bcd,cam_back)
o3d.visualization.draw_geometries([pcd,bcd])

In [20]:
merge = mergeTwoClouds(pcd,bcd)
o3d.visualization.draw_geometries([merge])

In [21]:
print('run Poisson surface reconstruction')
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(merge, depth=8)
print(mesh)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

run Poisson surface reconstruction
geometry::TriangleMesh with 3469 points and 6912 triangles.


In [22]:
o3d.io.write_triangle_mesh('./Mesh/Poisson Mesh/pose_1.ply',mesh)

True

temp = np.asarray(pcd.points)
new_points = duplicateSurface(temp)
new_pcl = o3d.geometry.PointCloud()
new_pcl.points = o3d.utility.Vector3dVector(np.concatenate((temp,new_points)))
new_pcl.normals = o3d.utility.Vector3dVector(np.concatenate((np.asarray(pcd.normals),-np.asarray(pcd.normals))))
new_pcl.paint_uniform_color([0.8, 0.8, 0.8])
#o3d.visualization.draw_geometries([new_pcl])
#o3d.io.write_point_cloud("./clean points/dupback_pose_1.ply", new_pcl)